In [2]:
import pandas as pd
import numpy as np
import sys
import plotly.graph_objects as go
sys.path.append('../../../')
from PIL import Image
import os

In [4]:
# Read data skipping the first row
df = pd.read_excel('../data/volcomm.xlsx', sheet_name='Domain Level Scores')

# Keep first 18 columns
df = df.iloc[:, :18]
df.columns


Index(['Commitment', 'Indicator', 'Adobe', 'Amazon', 'Anthropic', 'Apple',
       'Cohere', 'Google', 'IBM', 'Inflection', 'Meta', 'Microsoft', 'Nvidia',
       'OpenAI', 'Palantir', 'Salesforce', 'Scale AI', 'Stability AI'],
      dtype='object')

In [5]:
def add_line_breaks(text, max_length):
    """Insert line breaks into text to ensure it does not exceed max_length."""
    words = text.split()
    current_line = ""
    formatted_text = ""

    for word in words:
        # Check if adding the next word exceeds the max_length
        if len(current_line) + len(word) + 1 > max_length:
            formatted_text += current_line + "<br>"  # Add line break
            current_line = word  # Start a new line with the current word
        else:
            current_line += " " + word if current_line else word  # Add word to current line

    formatted_text += current_line  # Add any remaining text
    return formatted_text

# Apply the function to the indicator columns
df['Indicator'] = df['Indicator'].apply(lambda x: add_line_breaks(x, 100))  # Set max_length to 15

In [6]:
# convert columns to numeric except for 'Indicator' and 'Subdomain'
df[df.columns[2:]] = df[df.columns[2:]].apply(pd.to_numeric, errors = 'coerce')

# # Add a new row with column totals
df.loc['Total'] = df.sum(numeric_only=True, axis=0)
df.loc['Total', 'Indicator'] = '<b>Commitment Subtotal<b>'
df.loc['Total', 'Commitment'] = ''

In [7]:
df = df.iloc[::-1]
df = df.melt(id_vars=['Commitment', 'Indicator'], var_name='Company', value_name='score')

In [8]:
# Create a dictionary to map values in 'Subdomain' column to values in 'Indicator' column
subdomain_indicator = dict(zip(df.Commitment, df.Indicator))

# Create lists for 'Subdomain' and 'Indicator' columns
x_subdomain = list(df.Commitment)
x_indicator = list(df.Indicator)

# Create a list of tuples for 'Subdomain' and 'Indicator' columns
x_multicategory_subdomain = [x_subdomain, x_indicator]

# Convert 'score' values to strings
text=list(df.score)
text = [str(x) for x in text]

In [9]:
# Define a colorscale for the heatmap
# '#344E41
# '#588157'
# colorscale = [[0, '#DAD7CD'], [1, '#4C7048']]
colorscale = [[0, '#b5bea0'], [1, '#344E41']]

# Create a variable for font type, font style and font size
font = "CircularStd-Book"
font_bold = "CircularStd-Bold"
font_size = 22

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
                z=df.score,
                y=x_multicategory_subdomain,
                x=list(df.Company),
                hoverongaps=False,
                colorscale=colorscale,
                zmin=0,
                zmax=1,
                opacity=1,  # Changed from 0.5 to 1.0 for solid colors
                text=text,
                showscale=False,
                texttemplate="%{text:.0f}",
                textfont=dict(
                    family=font,
                    size=font_size
                ),
                xgap=0.01,  # Add gap between columns
                ygap=0.01   # Add gap between rows
))

fig.update_layout(
    height=500,
    # plot_bgcolor='white',  # Set plot background to white
    # paper_bgcolor='white',  # Set paper background to white
    yaxis=dict(
        ticklabelposition="inside",
        tickangle=0,
        automargin=True,
        gridcolor='white',  # Hide grid lines
        # showgrid=False,
        # title=dict(standoff=15)
    ),
    xaxis=dict(
        # showgrid=False,
        # gridcolor='white'  # Hide grid lines
    )
)

# Extract df.score values and calculate last row
last_row = list(df.score)[::len(set(x_multicategory_subdomain[1]))]
model_ind_count = len(set(x_multicategory_subdomain[1])) - 1
last_row = [x / model_ind_count for x in last_row]

# Add annotations and shapes for the last row
for i, c in enumerate(last_row):
    # Add white rectangle background
    fig.add_shape(
        type='rect',
        x0=i-0.5,
        y0=-0.5,
        x1=i+0.5,
        y1=0.5,
        line=dict(
            color='rgb(188,189,220)',
            width=1,
        ),
        fillcolor='white',
        layer='above'
    )
    # Add percentage annotations
    fig.add_annotation(
        x=i,
        y=0,
        text=f'{c:.0%}',
        showarrow=False,
        font=dict(
            family=font_bold,
            size=font_size,
            color="black"
        ),
    )
# Add vertical line that indicates the Domain
color = '#344E41'
fig.add_shape(type='line',
                x0=-0.5, y0=0.5, x1=-0.5, y1=+30.5,
                line=dict(
                    color=color,
                    width=7,
                ))

# Add horizontal divide line for each 'Indicator'
for i in range(0, len(set(x_multicategory_subdomain[1]))+1):
    fig.add_shape(type='line',
                x0=-0.5, y0=+i-0.5, x1=+15.5, y1=+i-0.5,
                line=dict(
                    color='gray',
                    width=0.7,
                ))
            
# Add annotations for 'Subdomain' and 'Indicator'
fig.add_annotation(
    x=-0.45,
    y=1.026,
    xref="paper",
    yref="paper",
    text="Commitment",
    showarrow=False,
    font=dict(
        family=font_bold,
        size=font_size,
        color="black"
    ),
)
fig.add_annotation(
    x=-0.05,
    y=1.026,
    xref="paper",
    yref="paper",
    text="Indicator",
    showarrow=False,
    font=dict(
        family=font_bold,
        size=font_size,
        color="black"
    ),
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'colorscale': [[0, '#b5bea0'], [1, '#344E41']],
              'hoverongaps': False,
              'opacity': 1,
              'showscale': False,
              'text': [13.0, 0.0, 0.0, ..., 0.0, 1.0, 1.0],
              'textfont': {'family': 'CircularStd-Book', 'size': 22},
              'texttemplate': '%{text:.0f}',
              'type': 'heatmap',
              'x': [Adobe, Adobe, Adobe, ..., Stability AI, Stability AI,
                    Stability AI],
              'xgap': 0.01,
              'y': [['', "Address society's greatest challenges", "Address
                    society's greatest challenges", "Address society's greatest
                    challenges", 'Societal risk research', 'Societal risk
                    research', 'Societal risk research', 'Societal risk research',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Watermarking/Provenance',
                    'Watermarking/Provenance', 'Watermarking/Provenance',
                    'Watermarking/Provenance', 'Third-party reporting', 'Third-
                    party reporting', 'Model weight security', 'Model weight
                    security', 'Model weight security', 'Information Sharing',
                    'Information Sharing', 'Information Sharing', 'Information
                    Sharing', 'Red-teaming', 'Red-teaming', 'Red-teaming', '',
                    "Address society's greatest challenges", "Address society's
                    greatest challenges", "Address society's greatest challenges",
                    'Societal risk research', 'Societal risk research', 'Societal
                    risk research', 'Societal risk research', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Watermarking/Provenance', 'Watermarking/Provenance',
                    'Watermarking/Provenance', 'Watermarking/Provenance', 'Third-
                    party reporting', 'Third-party reporting', 'Model weight
                    security', 'Model weight security', 'Model weight security',
                    'Information Sharing', 'Information Sharing', 'Information
                    Sharing', 'Information Sharing', 'Red-teaming', 'Red-teaming',
                    'Red-teaming', '', "Address society's greatest challenges",
                    "Address society's greatest challenges", "Address society's
                    greatest challenges", 'Societal risk research', 'Societal risk
                    research', 'Societal risk research', 'Societal risk research',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Watermarking/Provenance',
                    'Watermarking/Provenance', 'Watermarking/Provenance',
                    'Watermarking/Provenance', 'Third-party reporting', 'Third-
                    party reporting', 'Model weight security', 'Model weight
                    security', 'Model weight security', 'Information Sharing',
                    'Information Sharing', 'Information Sharing', 'Information
                    Sharing', 'Red-teaming', 'Red-teaming', 'Red-teaming', '',
                    "Address society's greatest challenges", "Address society's
                    greatest challenges", "Address society's greatest challenges",
                    'Societal risk research', 'Societal risk research', 'Societal
                    risk research', 'Societal risk research', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
                    'Public reporting', 'Public reporting', 'Public reporting',
        

In [10]:
# Load logos in png format
adobe_logo = Image.open('logos/adobe_logo.png')
apple_logo = Image.open('logos/apple_logo.png')
cohere_logo = Image.open('logos/cohere_logo2.png')
inflection_logo = Image.open('logos/inflection_logo2.png')
nvidia_logo = Image.open('logos/nvidia_logo.png')
palantir_logo = Image.open('logos/palantir_logo.png')
salesforce_logo = Image.open('logos/salesforce_logo.png')
scale_logo = Image.open('logos/scale_logo.png')
ibm_logo = Image.open('logos/ibm-logo-black-transparent.png')
meta_logo = Image.open('logos/meta_logo.png')
openai_logo = Image.open('logos/openai_logo.png')
microsoft_logo = Image.open('logos/microsoft_logo.png')
stabilityai_logo = Image.open('logos/stabilityai_logo.png')
anthropic_logo = Image.open('logos/anthropic_logo.png')
google_logo = Image.open('logos/google_logo.png')
amazon_logo = Image.open('logos/amazon_logo.png')

y_shift = 0.02

# Update layout with logos
fig.update_layout(
    images=[dict(
            source=adobe_logo,
            xref="paper", yref="paper",
            x=0.06, y=1.01 + y_shift,
            sizex=0.08, sizey=0.08,
            xanchor="right", yanchor="bottom"
        ),
        dict(
            source=amazon_logo,
            xref="paper", yref="paper",
            x=0.12, y=1.03 + y_shift,
            sizex=0.05, sizey=0.05,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=anthropic_logo,
            xref="paper", yref="paper",
            x=0.19, y=1.038 + y_shift,
            sizex=0.06, sizey=0.06,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=apple_logo,
            xref="paper", yref="paper",
            x=0.245, y=1.03 + y_shift,
            sizex=0.045, sizey=0.045,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=cohere_logo,
            xref="paper", yref="paper",
            x=0.306, y=1.035 + y_shift,
            sizex=0.06, sizey=0.06,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=google_logo,
            xref="paper", yref="paper",
            x=0.37, y=1.035 + y_shift,
            sizex=0.05, sizey=0.05,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=ibm_logo,
            xref="paper", yref="paper",
            x=0.435, y=1.035 + y_shift,
            sizex=0.045, sizey=0.045,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=inflection_logo,
            xref="paper", yref="paper",
            x=0.49, y=1.035 + y_shift,
            sizex=0.045, sizey=0.045,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=meta_logo,
            xref="paper", yref="paper",
            x=0.57, y=1.022 + y_shift,
            sizex=0.075, sizey=0.075,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=microsoft_logo,
            xref="paper", yref="paper",
            x=0.63, y=1.034 + y_shift,
            sizex=0.06, sizey=0.06,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=nvidia_logo,
            xref="paper", yref="paper",
            x=0.685, y=1.034 + y_shift,
            sizex=0.055, sizey=0.055,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=openai_logo,
            xref="paper", yref="paper",
            x=0.75, y=1.03 + y_shift,
            sizex=0.06, sizey=0.06,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=palantir_logo,
            xref="paper", yref="paper",
            x=0.81, y=1.02 + y_shift,
            sizex=0.05, sizey=0.05,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=salesforce_logo,
            xref="paper", yref="paper",
            x=0.865, y=1.02 + y_shift,
            sizex=0.045, sizey=0.045,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=scale_logo,
            xref="paper", yref="paper",
            x=0.93, y=1.03 + y_shift,
            sizex=0.045, sizey=0.045,
            xanchor="right", yanchor="bottom"
            ),
        dict(
            source=stabilityai_logo,
            xref="paper", yref="paper",
            x=1, y=1.03 + y_shift,
            sizex=0.06, sizey=0.06,
            xanchor="right", yanchor="bottom"
            )
        ],
    xaxis=dict(
        tickfont=dict(
            family=font,
            size=font_size,
            color='black'
        ),
        side = 'top'
    ),
    yaxis=dict(
        tickfont=dict(
            family=font,
            size=font_size,
            color='black'
        ),
        dividercolor='gray',
        dividerwidth=1
        ),
    plot_bgcolor='rgba(0,0,0,0)',
    margin = dict(
        l=10,
        r=10,
        b=10,
        t=190,
    ),
    autosize=False,
    height=1500,
    width=3300,
)

# Update y-axis type to 'multicategory'
fig.update_yaxes(type='multicategory')
fig.write_image(f'../visualizations/indicator_level_scores.pdf', scale=4/3)